# Example of extracting features from a pretrained neural network in Caffe using SkiCaffe

SkiCaffe is a wrapper that provides a "scikit-learn like" API to pretrained networks such as those distributed in the [Caffe Model Zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo) or elsewhere (such as [DeepDetect](http://www.deepdetect.com/applications/model/)). Basically, I wanted to use these pretrained models for extracting features, but also use the powerful pipelines of scikit-learn. Here we illustrate it's basic use for extracting features. 

In [1]:
from skicaffe import SkiCaffe

To use SKiCaffe, we need Caffe so we have to specify where Caffe was installed. We assume that the installation of Caffe is the default one from [BVLC](https://github.com/BVLC/caffe). We specify the location of Caffe on our system with caffe_root:

In [2]:
caffe_root = '/usr/local/caffe/'
model_file = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
prototxt_file = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'

DLmodel = SkiCaffe(caffe_root = caffe_root,
                   model_prototxt_path = prototxt_file, 
                   model_trained_path = model_file, 
                   include_labels = True,
                   return_type = "pandasDF")

In Scikit-learn parlance, SkiCaffe is an estimator, since it is meant for extracting features from images. Therefore SkiCaffe inherits from the [BaseEstimator](http://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html#sklearn.base.BaseEstimator) and [TransformerMixin](http://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html#sklearn.base.TransformerMixin) classes. The two methods that we overwrite are the fit and transform methods. SkiCaffe is a bit unusual in that the input data that is transformed is not a numpy array X of shape [n_samples, n_features], but rather the input is a python list of image paths. SkiCaffe takes these image paths and "transforms" them by returning features that are derived from a pretrained neural net in Caffe. 

The fit method loads the specified pretrained network. The transform method takes the list of images paths and returns the image features extracted from a specific layer as a numpy array (or optionally a Pandas Data Frame).  

To load the pretrained network, the fit method in SkiCaffe needs the paths for the following two files:
- deploy.prototxt file: neural network definition file for prediction
- caffemodel file: the trained neural network (for example, bvlc_googlenet.caffemodel)

The current setup of SkiCaffe is very ImageNet centric, therefore other supporting files from the default installation of Caffe are used (see documentation for more details and change as needed). 





## Load pretrained BVLC reference model

In this example we will be using the BVLC reference model and specify the protxt file and trained caffemodel file. We "fit" the model by loading the pretrained network. We can also use the layer_sizes attribute of our model to see what the different layers and their sizes are.

In [3]:
DLmodel.fit()
print 'Number of layers:', len(DLmodel.layer_sizes)
DLmodel.layer_sizes

caffe imported successfully
Number of layers: 15


[('data', (10, 3, 227, 227)),
 ('conv1', (10, 96, 55, 55)),
 ('pool1', (10, 96, 27, 27)),
 ('norm1', (10, 96, 27, 27)),
 ('conv2', (10, 256, 27, 27)),
 ('pool2', (10, 256, 13, 13)),
 ('norm2', (10, 256, 13, 13)),
 ('conv3', (10, 384, 13, 13)),
 ('conv4', (10, 384, 13, 13)),
 ('conv5', (10, 256, 13, 13)),
 ('pool5', (10, 256, 6, 6)),
 ('fc6', (10, 4096)),
 ('fc7', (10, 4096)),
 ('fc8', (10, 1000)),
 ('prob', (10, 1000))]

## Specify list of images and extract features

In [4]:
image_paths = ['./images/cat.jpg', 
               './images/1404329745.jpg']

We can now "transform" these images by extracting their features from our pretrained network. All we have to do is specify the layer name (listed above). Here we select the output of the last fully connected layer "fc8". If you like Data Frames (like me), then you can optionally specify that the return type be a Pandas Data Frame. 

In [5]:
image_features = DLmodel.transform(X = image_paths)
image_features.head()

,pred.class,pred.conf,fc8.0,fc8.1,fc8.2,fc8.3,fc8.4,fc8.5,fc8.6,fc8.7,...,fc8.990,fc8.991,fc8.992,fc8.993,fc8.994,fc8.995,fc8.996,fc8.997,fc8.998,fc8.999
0,"n02123045 tabby, tabby cat",0.301984,-3.951654,3.974992,-2.126802,-1.871261,-2.662160,-1.353720,-1.741305,0.325119,...,0.965206,-1.478447,-1.339074,-2.622084,-2.790083,-0.731835,-2.223104,-3.972739,3.412313,5.382214
1,n09472597 volcano,0.301601,-0.368596,-2.329629,-2.272709,-1.236161,0.337776,-1.146151,0.180243,-1.866235,...,1.175359,0.087868,-0.151791,-2.187589,-1.585882,1.581705,-1.362859,-0.552606,5.315578,-0.433723


Only get the features and use a different layer:

In [6]:
DLmodel.include_labels = True
image_features = DLmodel.transform(X = image_paths, layer_name='fc7')
image_features.head()

,pred.class,pred.conf,fc7.0,fc7.1,fc7.2,fc7.3,fc7.4,fc7.5,fc7.6,fc7.7,...,fc7.4086,fc7.4087,fc7.4088,fc7.4089,fc7.4090,fc7.4091,fc7.4092,fc7.4093,fc7.4094,fc7.4095
0,"n02123045 tabby, tabby cat",0.301984,0.0,0.0,0.0,0.0,0.0,0.0,6.55105,0.0,...,5.213141,4.056163,0.0,0.000000,0.0,1.66966,0.0,0.0,4.885226,6.546407
1,n09472597 volcano,0.301601,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.025197,0.0,3.830498,0.0,0.00000,0.0,0.0,0.000000,0.000000
